# Domácí úkol:
Pomocí vlastních tříd a dědičnosti vytvořte "soubor tříd" = nástroj pro derivaci funkcí.

## Jak to má fungovat:

- derivace je jednoduše algoritmizovatelná, neboť se řídí prakticky pouze třemi pravidly (podílu se dá vyhnout pomocí násobení funkcí $\frac{1}{f(x)}$):
    - derivace součtu je součet derivací
    - pravidlo pro derivaci součinu
    - derivace funkce je derivovaná funkce násobená derivovaným argumentem

- tedy stačí implementovat tyto tři základní pravidla a pro každou elementární funkci (sin, cos, exp, ln, x^n, ...) vědět jak vypadá její derivace
- celý tento proces je možné reprezentovat pomocí tříd a dědičnosti
- trik je v tom, že se výrazy automaticky skládají do stromu (pro nás v podstatě skrytého), a tedy každý výraz je na venek pouze buď sčítání, násobení, nebo elementární funkce (a o jejich argumentech platí totéž)

## Podrobnější zadání:
Konkrétněji:
- Vytvořte třídu **Operator**
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor (zde stačí `pass`)
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
- Vytvořte třídu **BinaryOperator**, která bude dědit od třídy Operator
    - Třída bude mít atributy:
        - `levy_argument` - levý operand
        - `pravy_argument` - pravý operand
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor, přiřadí argumenty
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
        - `diff()` - vrací derivaci operátoru
- Vytvořte třídu **UnaryOperator**, která bude dědit od třídy Operator
    - Třída bude mít atributy:
        - `argument` - argument funkce 
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor, přiřadí argument
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
        - `diff()` - vrací derivaci operátoru

Pro počítání derivací použijte standardní pravidla:
- derivace součtu je součet derivací
- pravidlo pro derivaci součinu
- derivace funkce je derivovaná funkce násobená derivovaným argumentem

Pro **BinaryOperator** a **UnaryOperator** implementujte také metody implementující standardní operátory:
- `__add__()` - implementuje operátor `+`
- `__sub__()` - implementuje operátor `-`
- `__mul__()` - implementuje operátor `*`

Pro **BinaryOperator** implementujte dvě dceřiné třídy:
- **Addition** - implementuje operátor `+`
- **Multiplication** - implementuje operátor `*`

Pro **UnaryOperator** implementujte tyto dceřiné třídy:
- **exp** - implementuje funkci `exp`
- **sin** - implementuje funkci `sin`
- **cos** - implementuje funkci `cos`
- **ln** - implementuje funkci `ln`
- **frc** - implementuje funkci `1/x`
- **identity** - implementuje identitu `x`
- **cst** - implementuje konstantu `c`
- **mocnina** - implementuje mocninu `x^n`

U všech funckí implementujte adekvátně metodu `diff()` a `__str__()`, tak aby se funkce vypisovaly v přehledné podobě.

Ozkoušejte na následujících funkcích:
- $sin(x + cos(x*x)) + 1$
- $x^2 + 2x + 1 + 2x^2$
- $x^2 + 2x + exp(x)$

In [359]:
# kostra k doplnění, místa označená TODO doplňte

class Operator:
    def __init__(self):
        pass
    
    def __str__(self):
        return type(self).__name__
    
    def __repr__(self):
        return self.__str__()

    def diff(self):
        pass

    def simplify(self):
        pass

    def __add__(self, other):
        return Addition(self, other)
    
    def __radd__(self, other):
        other = cst(other)
        return other + self
    
    def __sub__(self, other):
        return Addition(self, cst(-1) * other)
    
    def __rsub__(self, other):
        other = cst(other)
        return other - self
    
    def __mul__(self, other):
        return Multiplication(self, other)
    
    def __rmul__(self, other):
        other = cst(other)
        return self * other
    
class BinaryOperator(Operator):
    def __init__(self, levy_argument, pravy_argument):
        # Tahle kontrola praveho argumentu je potreba, protoze jinak by se do stromu funkci dotaly konstanty, ktere nejsou spravneho typu.
        # Byt to nejspis neni hezke reseni, nic lepsiho me nenapada.
        if(type(pravy_argument) == float or type(pravy_argument) == int):
            pravy_argument = cst(pravy_argument)

        self.levy_argument = levy_argument
        self.pravy_argument = pravy_argument
        
class UnaryOperator(Operator):
    def __init__(self, argument):
        if(type(argument) == float or type(argument) == int):
            argument = cst(argument)

        self.argument = argument
    
    def __str__(self):
        return type(self).__name__ + "(" + str(self.argument) + ")"\


class Addition(BinaryOperator):
    def __str__(self):
        return f"{self.levy_argument} + {self.pravy_argument}"
    
    def __eq__(self, other):
        if(type(other) !=  Addition):
            return False
        
        return self.levy_argument == other.levy_argument and self.pravy_argument == other.pravy_argument
    
    def diff(self):
        return self.levy_argument.diff() + self.pravy_argument.diff()

    def simplify(self):
        if(type(self.levy_argument) == type(self.pravy_argument) == cst):
            return cst(self.levy_argument.value + self.pravy_argument.value)

        if(type(self.levy_argument) == cst):
            if(self.levy_argument.value == 0):
                return self.pravy_argument.simplify()
        
        if(type(self.pravy_argument) == cst):
            if(self.pravy_argument.value == 0):
                return self.levy_argument.simplify()
            
        if(self.levy_argument == self.pravy_argument):
            return cst(2) * self.levy_argument.simplify()
        
        return self.levy_argument.simplify() + self.pravy_argument.simplify()
    
class Multiplication(BinaryOperator):
    def __str__(self):
        return f"({self.levy_argument}) * ({self.pravy_argument})"
        
    def __eq__(self, other):
        if(type(other) !=  Multiplication):
            return False
        
        return self.levy_argument == other.levy_argument and self.pravy_argument == other.pravy_argument
        
    def diff(self):
        return (self.levy_argument.diff() * self.pravy_argument) + (self.levy_argument * self.pravy_argument.diff())
    
    def simplify(self):
        if(type(self.levy_argument) == type(self.pravy_argument) == cst):
            return cst(self.levy_argument.value * self.pravy_argument.value)
        
        if(type(self.levy_argument) == cst):
            if(self.levy_argument.value == 0):
                return cst(0)
            if(self.levy_argument.value == 1):
                return self.pravy_argument.simplify()
            
        if(type(self.pravy_argument) == cst):
            if(self.pravy_argument.value == 0):
                return cst(0)
            if(self.pravy_argument.value == 1):
                return self.levy_argument.simplify()
            
            temp = self.levy_argument
            self.levy_argument = self.pravy_argument
            self.pravy_argument = temp

        if(self.levy_argument == self.pravy_argument):
            return mocnina(self.levy_argument.simplify(), cst(2))
        
        return self.levy_argument.simplify() * self.pravy_argument.simplify()
    
    def is_negative(self):
        if(type(self.levy_argument) == cst and self.levy_argument.value == -1):
            return True
        
        if(type(self.pravy_argument) == cst and self.pravy_argument.value == -1):
            return True
        
        return False

class cst(UnaryOperator):
    def __init__(self, value):
        self.value = value

    def __str__(self):
        return f"{self.value}"
                
    def __eq__(self, other):
        if(type(other) != cst):
            return False
        
        return self.value == other.value
    
    def diff(self):
        return cst(0)
    
    def simplify(self):
        return self
    
class identity(UnaryOperator):
    def __init__(self):
        pass

    def __str__(self):
        return "x"
            
    def __eq__(self, other):
        return type(other) == identity
    
    def diff(self):
        return cst(1)
    
    def simplify(self):
        return self

class mocnina(UnaryOperator):
    def __init__(self, argument, exponent):
        if(type(exponent) == float or type(exponent) == int):
            exponent = cst(exponent)
            
        self.argument = argument
        self.exponent = exponent

    def __str__(self):
        return f"({self.argument})^{self.exponent}"
            
    def __eq__(self, other):
        if(type(other) != mocnina):
            return False
        
        return self.exponent == other.exponent and self.argument == other.argument
    
    def diff(self):
        return self.exponent * mocnina(self.argument, self.exponent - cst(1)) * self.argument.diff()
    
    def simplify(self):
        if(type(self.exponent) == cst):
            if(self.exponent.value == 0):
                return cst(1)
            if(self.exponent.value == 1):
                return self.argument.simplify()
            if(self.exponent.value == -1):
                return frc(mocnina(self.argument.simplify(), cst(-1 * self.exponent.value)))
            
        return mocnina(self.argument.simplify(), self.exponent.simplify())
    
class sin(UnaryOperator):    
    def __eq__(self, other):
        if(type(other) != sin):
            return False
        
        return self.argument == other.argument
    
    def diff(self):
        return cos(self.argument) * self.argument.diff()
    
    def simplify(self):
        return sin(self.argument.simplify())

class cos(UnaryOperator):    
    def __eq__(self, other):
        if(type(other) != cos):
            return False
        
        return self.argument == other.argument
    
    def diff(self):
        return cst(-1) * sin(self.argument) * self.argument.diff()
    
    def simplify(self):
        return cos(self.argument.simplify())

    
class exp(UnaryOperator):    
    def __eq__(self, other):
        if(type(other) != exp):
            return False
        
        return self.argument == other.argument
    
    def diff(self):
        return self * self.argument.diff()
    
    def simplify(self):
        if(type(self.argument) == ln):
            return self.argument.simplify()
        
        if(type(self.argument) == cst):
            if(self.argument.value == 0):
                return cst(1)
        
        return exp(self.argument.simplify())

class ln(UnaryOperator):
    def __eq__(self, other):
        if(type(other) != ln):
            return False
        
        return self.argument == other.argument
    
    def diff(self):
        return frc(self.argument) * self.argument.diff()
    
    def simplify(self):
        if(type(self.argument) == exp):
            return self.argument.simplify()
        
        return ln(self.argument.simplify())
    
class frc(UnaryOperator):
    def __str__(self):
        return f"1/{self.argument}"    
    
    def __eq__(self, other):
        if(type(other) != frc):
            return False
        
        return self.argument == other.argument
    
    def diff(self):
        return cst(-1) * frc(mocnina(self.argument, cst(2))) * self.argument.diff()
    
    def simplify(self):
        return frc(self.argument.simplify())
    


In [349]:
# takto by se to mělo používat
x = identity()
# sin(x + cos(x*x)) + 1
f1 = sin(x + cos(x*x)) + 1
print(f1.simplify())
print(f1.diff().simplify().simplify().simplify().simplify().simplify())


sin(x + cos((x)^2)) + 1
(cos(x + cos((x)^2))) * (1 + ((-1) * (sin((x)^2))) * ((2) * (x)))


můj výstup byl:

`sin(x + cos((x)*(x))) + 1`

`(cos(x + cos((x)*(x))))*(1 + ((-1)*(sin((x)*(x))))*((1)*(x) + (x)*(1))) + 0`

In [350]:
x = identity()
# x ^ 2 + 2x + 1 + 2x ^ 2
f2 = mocnina(x, 2) + 2 * x + 1 + 2 * mocnina(x, 2)
print(f2.simplify())
print(f2.diff().simplify().simplify().simplify().simplify())


(x)^2 + (2) * (x) + 1 + (2) * ((x)^2)
(2) * (x) + 2 + (2) * ((2) * (x))


můj výstup byl:

`(x)^2 + (2)*(x) + 1 + (2)*((x)^2)`

`((2)*((x)^1))*(1) + (0)*(x) + (2)*(1) + 0 + (0)*((x)^2) + (2)*(((2)*((x)^1))*(1))`

In [366]:
x = identity()
# x ^ 2 + 2x + exp(x)
f3 = mocnina(x,2) + 2*x + exp(x)
print(f3.simplify())
print(f3.diff().simplify().simplify().simplify())


(x)^2 + (2) * (x) + exp(x)
(2) * (x) + 2 + exp(x)


můj výstup byl:

`(x)^2 + (2)*(x) + exp(x)`

`((2)*((x)^1))*(1) + (0)*(x) + (2)*(1) + (exp(x))*(1)`


# Bonusový úkol:
Doplňte ke všem třídám metodu `simplify()`, která provede zjednodušení výrazu. Například:
- součet s nulou vrátí druhý operand
- součin s nulou vrátí nulu
- mocnina s exponentem 0 vrátí 1
- mocnina s exponentem 1 vrátí první argument
- součín dvou stejných argumentů vrátí mocninu s exponentem 2
- součin konstant vrátí konstantu se součinem
- ...

Doplňte metody `__add__, __radd__, __mul__, __rmul__, ...` tak aby podporovaly operace s čísly (např. `2 + x` nebo `x * 3`), a zjednodušovaly zadávání výrazů. 